In [ ]:
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.static import teams
from nba_api.stats.endpoints import boxscoreadvancedv2

from time import sleep
import datetime

import pandas as pd
import numpy as np

In [ ]:
team_dict = teams.get_teams()

In [ ]:
df = pd.DataFrame()
for season in range(2007,2022): 
    print(season)
    for team in team_dict:
        #print(team["full_name"])
        gamelogobj = teamgamelog.TeamGameLog(team_id = str(team["id"]), season = season)
        gamelogdf = gamelogobj.get_data_frames()[0]
        df = pd.concat([df, gamelogdf])
        sleep(.600) #Need to pause between loops so no timeout occurs

In [ ]:
games = df["Game_ID"].unique()
adv_df = pd.DataFrame()
i = 0
game_errors = []
for game in games:
    if i %100 == 0:
        print(i)
    i+= 1
    try:
        
        bsa = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id = str(game))
        bsadf = bsa.get_data_frames()[1]
        adv_df = pd.concat([adv_df,bsadf])
    except:
        print("ERROR:      " + str(i-1) + "     game: 00" + str(game)) #If there is an error, print out which game caused it
        game_errors.append(str(game))
    sleep(.7)

In [ ]:
#If there are errors, run code again to add the missing games

for game in game_errors:
    try:       
        bsa = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id = str(game))
        bsadf = bsa.get_data_frames()[1]
        adv_df = pd.concat([adv_df,bsadf])
    except:
        print("ERROR:      " + str(i-1) + "     game: 00" + str(game)) #If there is an error, print out which game caused it
    sleep(.7)

In [ ]:
df = df.reset_index()
df.rename(columns = {'index': "GAMES_LEFT"}, inplace = True)

In [ ]:
#Add identifying features 

df["SEASON"] = df["Game_ID"].apply(lambda x: str(x)[3:5])
df["Home"] = df.MATCHUP.apply(lambda x: 0 if x.__contains__('@') else 1)
df["Team"] = df.MATCHUP.apply(lambda x: x.split(" ")[0])
df["Opp"] = df.MATCHUP.apply(lambda x: x.split(" ")[2])

In [ ]:
#Sum all values from previous games for that particular team in that specific season

#Values they had
df["FGMiss"] = df[["FGA", "FGM"]].apply(lambda x: x["FGA"] - x["FGM"], axis =1)
df["Tot_Prev_Points"]= df[["Game_ID", "SEASON", "Team_ID"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Team_ID"] ==x["Team_ID"]), "PTS"].sum(), axis = 1)
df["Tot_Prev_Ast"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Team"] ==x["Team"]), "AST"].sum(), axis = 1)
df["Tot_OREB"] = df[["Game_ID", "SEASON", "Team_ID"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Team_ID"] ==x["Team_ID"]), "OREB"].sum(), axis = 1)
df["Tot_TOV"] = df[["Game_ID", "SEASON", "Team_ID"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Team_ID"] ==x["Team_ID"]), "TOV"].sum(), axis = 1)
df["Tot_FGMiss"] = df[["Game_ID", "SEASON", "Team_ID"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Team_ID"] ==x["Team_ID"]), "FGMiss"].sum(), axis = 1)
df["Tot_FGMade"] = df[["Game_ID", "SEASON", "Team_ID"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Team_ID"] ==x["Team_ID"]), "FGM"].sum(), axis = 1)
df["Tot_FGA"] = df[["Game_ID", "SEASON", "Team_ID"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Team_ID"] ==x["Team_ID"]), "FGA"].sum(), axis = 1)
df["Tot_FTM"] = df[["Game_ID", "SEASON", "Team_ID"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Team_ID"] ==x["Team_ID"]), "FTM"].sum(), axis = 1)
df["Tot_FTA"] = df[["Game_ID", "SEASON", "Team_ID"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Team_ID"] ==x["Team_ID"]), "FTA"].sum(), axis = 1)

In [ ]:
#Add the values the team gave up in the game to each row

df["Opp_PTS"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] == x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "PTS"].sum(), axis = 1)
df["Opp_OREB"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] == x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "OREB"].sum(), axis = 1)
df["Opp_DREB"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] == x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "DREB"].sum(), axis = 1)                                                     
df["Opp_FGMiss"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] == x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "FGMiss"].sum(), axis = 1)
df["Opp_FGMade"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] == x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "FGM"].sum(), axis = 1)
df["Opp_TOV"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] == x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "TOV"].sum(), axis = 1)
df["Opp_AST"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] == x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "AST"].sum(), axis = 1)
df["Opp_FG3M"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] == x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "FG3M"].sum(), axis = 1)
df["Opp_FGA"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] == x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "FGA"].sum(), axis = 1)                                                        
df["Opp_FTA"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] == x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "FTA"].sum(), axis = 1)                                                        


In [ ]:
#Sum the total values a team allowed

df["Tot_Opp_Prev_Points"]= df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "PTS"].sum(), axis = 1)
df["Tot_Opp_OREB"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "OREB"].sum(), axis = 1)
df["Tot_Opp_FGMiss"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "FGMiss"].sum(), axis = 1)
df["Tot_Opp_FGMade"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "FGM"].sum(), axis = 1)
df["Tot_Opp_TOV"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "TOV"].sum(), axis = 1)
df["Tot_Opp_AST"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "AST"].sum(), axis = 1)
df["Tot_Opp_FGA"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "FGA"].sum(), axis = 1)
df["Tot_Opp_FG3M"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "FG3M"].sum(), axis = 1)
df["Tot_Opp_FTA"] = df[["Game_ID", "SEASON", "Team"]].apply(lambda x: df.loc[(df["Game_ID"] < x["Game_ID"]) & (df["SEASON"] == x["SEASON"]) & (df["Opp"] ==x["Team"]), "FTA"].sum(), axis = 1)

In [ ]:
#Create Features to sum previous possesions

adv_df["SEASON"] =adv_df["GAME_ID"].apply(lambda x: str(x)[3:5])
adv_df["OPP_ID"] = adv_df[["GAME_ID", "TEAM_ID"]].apply(lambda x: adv_df.loc[(adv_df["GAME_ID"] == x["GAME_ID"]) & (adv_df["TEAM_ID"] != x["TEAM_ID"]), "TEAM_ID"].values[0], axis = 1)
adv_df["PREV_TOT_POSS"] = adv_df[["TEAM_ID" , "GAME_ID", "SEASON"]].apply(lambda x: adv_df.loc[(adv_df["TEAM_ID"] == x["TEAM_ID"]) & (adv_df["GAME_ID"]<x["GAME_ID"]) & (adv_df["SEASON"] == x["SEASON"]), "POSS"].sum(), axis = 1)
adv_df["PREV_OPP_TOT_POSS"] = adv_df[["TEAM_ID" , "GAME_ID", "SEASON"]].apply(lambda x: adv_df.loc[(adv_df["OPP_ID"] == x.TEAM_ID) & (adv_df["GAME_ID"]<x["GAME_ID"]) & (adv_df["SEASON"] == x["SEASON"]), "POSS"].sum(), axis = 1)  

In [ ]:
#Copy over values into main dataframe

df["Poss"] = df[["Game_ID", "Team_ID"]].apply(lambda x: adv_df.loc[(adv_df["GAME_ID"] == x["Game_ID"]) & (adv_df["TEAM_ID"] == x["Team_ID"]), "POSS"].sum(), axis = 1)
df["Tot_Prev_Poss"] = df[["Game_ID", "Team_ID"]].apply(lambda x: adv_df.loc[(adv_df["GAME_ID"] == x["Game_ID"]) & (adv_df["TEAM_ID"] == x["Team_ID"]), "PREV_TOT_POSS"].sum(), axis = 1)

df["Opp_Poss"] = df[["Game_ID", "Team_ID"]].apply(lambda x: adv_df.loc[(adv_df["GAME_ID"] == x["Game_ID"]) & (adv_df["OPP_ID"] == x["Team_ID"]), "POSS"].sum(), axis = 1)
df["Tot_Opp_Prev_Poss"] = df[["Game_ID", "Team_ID"]].apply(lambda x: adv_df.loc[(adv_df["GAME_ID"] == x["Game_ID"]) & (adv_df["TEAM_ID"] == x["Team_ID"]), "PREV_OPP_TOT_POSS"].sum(), axis = 1)

In [ ]:
#Calculate the rating values

#Values they had
df["ORTG"] = df[["Tot_Prev_Poss", "Tot_Prev_Points"]].apply(lambda x: x["Tot_Prev_Points"] / x["Tot_Prev_Poss"] * 100, axis = 1)
df["OREBRTG"] = df[["Tot_FGMiss", "Tot_OREB"]].apply(lambda x: x["Tot_OREB"] / x["Tot_FGMiss"] * 100, axis = 1)
df["TOVRTG"] = df[["Tot_Prev_Poss", "Tot_TOV"]].apply(lambda x: x["Tot_TOV"] / x["Tot_Prev_Poss"] * 100, axis = 1)
df["ASTRTG"] = df[["Tot_FGMade", "Tot_Prev_Ast"]].apply(lambda x: x["Tot_Prev_Ast"] / x["Tot_FGMade"] *100 , axis = 1)

#Values they allowed
df["DRTG"] = df[["Tot_Opp_Prev_Poss", "Tot_Opp_Prev_Points"]].apply(lambda x: x["Tot_Opp_Prev_Points"] / x["Tot_Opp_Prev_Poss"] * 100, axis = 1)
df["DTOVRTG"] = df[["Tot_Opp_Prev_Poss", "Tot_Opp_TOV"]].apply(lambda x: x["Tot_Opp_TOV"] / x["Tot_Opp_Prev_Poss"] * 100, axis = 1)
df["DREBRTG"] = df[["Tot_Opp_FGMiss", "Tot_Opp_OREB"]].apply(lambda x: x["Tot_Opp_OREB"] / x["Tot_Opp_FGMiss"] * 100, axis = 1)
df["DASTRTG"] = df[["Tot_Opp_FGMade", "Tot_Opp_AST"]].apply(lambda x: x["Tot_Opp_AST"] / x["Tot_Opp_FGMade"] * 100, axis = 1)

In [ ]:
#Verify Values

df[(df["SEASON"] == "21") & (df["GAMES_LEFT"] == 0)][["Team","Opp", "Poss", "Tot_Prev_Poss", "Tot_Prev_Points", "PTS", "ORTG"]]

In [ ]:
#All null values are due to first game of season, therefore set those values as 0

df=df.fillna(0)

In [ ]:
#Split all values into home and away to prepare for grouping
#If a team is the home team, set home team values as the team's values else set as 0
#If a team is the away team, set away team values as the team's values else set as 0

df['Home_ORTG'] = df[["Home", "ORTG"]].apply(lambda x: x["ORTG"] if x["Home"] == 1 else 0, axis = 1)
df['Away_ORTG'] = df[["Home", "ORTG"]].apply(lambda x: x["ORTG"] if x["Home"] == 0 else 0, axis = 1)
df['Home_DRTG'] = df[["Home", "DRTG"]].apply(lambda x: x["DRTG"] if x["Home"] == 1 else 0, axis = 1)
df['Away_DRTG'] = df[["Home", "DRTG"]].apply(lambda x: x["DRTG"] if x["Home"] == 0 else 0, axis = 1)
df['Home_OREBRTG'] = df[["Home", "OREBRTG"]].apply(lambda x: x["OREBRTG"] if x["Home"] == 1 else 0, axis = 1)
df['Away_OREBRTG'] = df[["Home", "OREBRTG"]].apply(lambda x: x["OREBRTG"] if x["Home"] == 0 else 0, axis = 1)
df['Home_DREBRTG'] = df[["Home", "DREBRTG"]].apply(lambda x: x["DREBRTG"] if x["Home"] == 1 else 0, axis = 1)
df['Away_DREBRTG'] = df[["Home", "DREBRTG"]].apply(lambda x: x["DREBRTG"] if x["Home"] == 0 else 0, axis = 1)
df['Home_TOVRTG'] = df[["Home", "TOVRTG"]].apply(lambda x: x["TOVRTG"] if x["Home"] == 1 else 0, axis = 1)
df['Away_TOVRTG'] = df[["Home", "TOVRTG"]].apply(lambda x: x["TOVRTG"] if x["Home"] == 0 else 0, axis = 1)
df['Home_DTOVRTG'] = df[["Home", "DTOVRTG"]].apply(lambda x: x["DTOVRTG"] if x["Home"] == 1 else 0, axis = 1)
df['Away_DTOVRTG'] = df[["Home", "DTOVRTG"]].apply(lambda x: x["DTOVRTG"] if x["Home"] == 0 else 0, axis = 1)
df['Home_ASTRTG'] = df[["Home", "ASTRTG"]].apply(lambda x: x["ASTRTG"] if x["Home"] == 1 else 0, axis = 1)
df['Away_ASTRTG'] = df[["Home", "ASTRTG"]].apply(lambda x: x["ASTRTG"] if x["Home"] == 0 else 0, axis = 1)
df["Home_Team"] = df[["Home", "Team", "Opp"]].apply(lambda x: x["Team"] if x["Home"] == 1 else x["Opp"], axis = 1)
df["Away_Team"] = df[["Home", "Team", "Opp"]].apply(lambda x: x["Team"] if x["Home"] == 0 else x["Opp"], axis = 1)
df['Home_DASTRTG'] = df[["Home", "DASTRTG"]].apply(lambda x: x["DASTRTG"] if x["Home"] == 1 else 0, axis = 1)
df['Away_DASTRTG'] = df[["Home", "DASTRTG"]].apply(lambda x: x["DASTRTG"] if x["Home"] == 0 else 0, axis = 1)

In [ ]:
#Point spread for each game

df["Spread"] = df[["Home", "PTS"]].apply(lambda x: -x["PTS"] if x["Home"] == 0 else x["PTS"], axis = 1)

In [ ]:
#Whether the home team won (1) or lost (0)

df["Home_Win"] = df[["WL", "Home"]].apply(lambda x: 1 if ((x["Home"] == 1) & (x["WL"] == "W")) else 0, axis = 1)

In [ ]:
#Calculating whether the team played the day before

df["GAME_DATE"] = df["GAME_DATE"].apply(lambda x: datetime.datetime.strptime(x,"%b %d, %Y"))
df["Back_To_Back"] = df[["GAME_DATE", "Team_ID"]].apply(lambda x: 1 if (x["GAME_DATE"] - datetime.timedelta(1)) in df.loc[(df["Team_ID"] == x["Team_ID"]), "GAME_DATE"].values else 0, axis = 1)
df["Home_Team_B2B"] = df[["Back_To_Back", "Home"]].apply(lambda x: 1 if (x["Back_To_Back"] == 1) & (x["Home"] == 1) else 0, axis =1 )
df["Away_Team_B2B"] = df[["Back_To_Back", "Home"]].apply(lambda x: 1 if (x["Back_To_Back"] == 1) & (x["Home"] == 0) else 0, axis =1 )

In [ ]:
#Group by game ID and sum all values

grouped_df = df.groupby(by = ["Game_ID", "Home_Team", "Away_Team"]).sum()

In [ ]:
#Drop All unnecessary columns 

drop = ["Team_ID", "W", "L", "W_PCT", "MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT", 
        "FTM", "FTA", "FT_PCT", "REB", "OREB", "DREB",
        "AST", "STL", "BLK","TOV", "PF", "PTS", "Home", "Tot_Prev_Poss", "Back_To_Back",
        "Tot_Prev_Points", "Tot_Opp_Prev_Points", "ORTG", "Tot_Opp_Prev_Poss", "DRTG", "Tot_OREB", 
        "Tot_Opp_OREB","OREBRTG", "FGMiss", "Tot_FGMiss",
        "Tot_Opp_FGMiss", "DREBRTG","Tot_TOV", "Tot_Opp_TOV",
        "TOVRTG", "DTOVRTG", "Tot_Prev_Ast", "Tot_FGMade", "ASTRTG",
        'Opp_PTS', 'Opp_OREB', 'Tot_FGA', 'Tot_FTM', 'Tot_FTA',
        'Opp_FGMiss', 'Opp_FGMade', 'Opp_TOV', 'Opp_AST', 'Opp_FG3M', 'Opp_FGA',
        'Tot_Opp_FGMade', 'Tot_Opp_AST', 'Tot_Opp_FGA', 'Tot_Opp_FG3M',
        'Tot_Opp_FTA', 'Opp_FTA', 'Poss', 'Opp_DREB', 'Opp_Poss', 'DASTRTG']

test_df = grouped_df.drop([], axis = 1)

In [ ]:
test_df.reset_index(inplace = True)

test_df["Season"] = test_df["Game_ID"].apply(lambda x: str(x)[3:5])

In [ ]:
#Save complete data
test_df.to_csv("DataForNBAGameModelComplete.csv")

In [ ]:
#Create and save data excluding 2021 season and games where the teams haven't combined for 20 games played for the season
slimmed_df = test_df[(test_df["GAMES_LEFT"] < 143) & (test_df["Season"] != "21")]

slimmed_df.to_csv("DataForNBAGameModel0720.csv")

In [ ]:
#Create and save data excluding 2019-2021 seasons and games where the teams haven't combined for 20 games played for the season
slimmed_df0718 =  test_df[(test_df["GAMES_LEFT"] < 143) & 
                          (test_df["Season"] != "21") & 
                          (test_df["Season"] != "20") & 
                          (test_df["Season"] != "19")]

slimmed_df0718.to_csv("DataForNBAGameModel0718.csv")

In [ ]:
#Save data for just 2021 season
season21 = test[test_df["Season"] == "21"]

season21.to_csv("2021GameLogsForModel.csv")